In [1]:
import numpy as np
import pandas as pd

In [2]:
def get_attrib_name(df):
    return df.iloc[0,0] + '_' + df.iloc[0,1]

In [3]:
france_gdp = pd.read_csv('./DP_LIVE_06052020000825675.csv', usecols=[1, 3, 5, 6])

france = france_gdp.rename(columns={'Value': get_attrib_name(france_gdp)})
france = france.drop(france.columns[:2], axis='columns')
france = france.set_index('TIME', drop=True)

In [4]:
france_unemployment = pd.read_csv('./DP_LIVE_06052020002104118.csv', usecols=[1, 3, 5, 6])

france_unemployment = france_unemployment.rename(columns={'Value': get_attrib_name(france_unemployment)})
france_unemployment = france_unemployment.drop(france_unemployment.columns[:2], axis='columns')
france_unemployment = france_unemployment.set_index('TIME', drop=True)

In [5]:
france_hcpi = pd.read_csv('./ei_cphi_m_1_Data.csv')

quarterly_indices = france.index

def make_monthly_hcpi_index(mi):
    year, quarter = mi.split('-')
    month = str(int(quarter[-1])*3)
    if len(month) < 2:
        month = '0' + month
    return f'{year}M{month}'

monthly_indices_hcpi = pd.Index([make_monthly_hcpi_index(ix) for ix in quarterly_indices])

In [6]:
france_hcpi = france_hcpi.set_index('TIME', drop=True)
france_hcpi = france_hcpi.loc[monthly_indices_hcpi, ['Value']]
france_hcpi.rename(columns={'Value':'HCPI_(2010=100)'}, inplace=True)
france_hcpi.set_index(france.index, inplace=True)

/home/lukas/anaconda3/envs/handsonml/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [7]:
france_ave_wage = pd.read_csv('DP_LIVE_06052020011457728.csv', usecols=[1, 3, 5, 6])

france_ave_wage = france_ave_wage.rename(columns={'Value': get_attrib_name(france_ave_wage)})
france_ave_wage = france_ave_wage.drop(france_ave_wage.columns[:2], axis='columns')
france_ave_wage = france_ave_wage.set_index('TIME', drop=True)

In [8]:
france_ave_wage

,AVWAGE_USD
TIME,
2014,43308.845874
2015,43731.656865
2016,44220.254627
2017,44469.020578
2018,44510.133176


In [9]:
france_ave_wage_list = []

for year in france_ave_wage.index:
    for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
        make_new_ix = str(year) + '-' + quarter
        france_ave_wage_list.append({
            'TIME': make_new_ix,
            'AVWAGE_USD': france_ave_wage.loc[year, 'AVWAGE_USD']
        })
        
    
france_ave_wage = pd.DataFrame(france_ave_wage_list)
france_ave_wage = france_ave_wage.set_index('TIME', drop=True)

In [10]:
pd.DataFrame(france_ave_wage_list)

,TIME,AVWAGE_USD
0,2014-Q1,43308.845874
1,2014-Q2,43308.845874
2,2014-Q3,43308.845874
3,2014-Q4,43308.845874
4,2015-Q1,43731.656865
5,2015-Q2,43731.656865
6,2015-Q3,43731.656865
7,2015-Q4,43731.656865
8,2016-Q1,44220.254627
9,2016-Q2,44220.254627


In [11]:
france = pd.merge(france, france_unemployment, left_index=True, right_index=True)

In [12]:
france = pd.merge(france, france_hcpi, left_index=True, right_index=True)

In [13]:
france = pd.merge(france, france_ave_wage, left_index=True, right_index=True, how='outer')

In [14]:
france

,QGDP_PC_CHGPP,UNEMP_PC_LF,HCPI_(2010=100),AVWAGE_USD
TIME,,,,
2014-Q1,0.076965,10.160150,100.21,43308.845874
2014-Q2,0.166879,10.195080,100.16,43308.845874
2014-Q3,0.476135,10.321200,99.89,43308.845874
2014-Q4,0.099598,10.491620,99.82,43308.845874
2015-Q1,0.455249,10.309580,100.17,43731.656865
2015-Q2,-0.052567,10.475670,100.49,43731.656865
2015-Q3,0.425742,10.368470,99.99,43731.656865
2015-Q4,0.216838,10.262680,100.09,43731.656865
2016-Q1,0.647457,10.223360,100.09,44220.254627


In [15]:
france.to_excel('airi.xlsx')